In [29]:
import logging

# Configure the root logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

from datasets import load_dataset # huggingface datasets
# takes 5GB in huggingface .cache dir, about 727k documents
# load only val dataset for quick testing
dataset = load_dataset("json", data_files="./data/json/val.json")
# split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=2357, shuffle=True)
# split_dataset['val'] = split_dataset.pop('test')  # rename the test split to val
# split_dataset['train']

2023-05-04 22:12:10,068 [WARNING] Found cached dataset json (/Users/make/.cache/huggingface/datasets/json/default-36fa927edf95b427/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

2023-05-04 22:12:10,080 [WARNING] Loading cached split indices for dataset at /Users/make/.cache/huggingface/datasets/json/default-36fa927edf95b427/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-af7767bbbd076ab8.arrow and /Users/make/.cache/huggingface/datasets/json/default-36fa927edf95b427/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-5a76445b9a03c715.arrow


Dataset({
    features: ['labels', 'id', 'sentences'],
    num_rows: 65118
})

In [30]:
from sentence_transformers import SentenceTransformer
enc = SentenceTransformer('all-MiniLM-L6-v2')
import logging


2023-05-04 22:12:10,627 [INFO] Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-05-04 22:12:10,983 [INFO] Use pytorch device: cpu


In [ ]:
import torch
torch.set_default_device("mps")
from datasets import load_dataset

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, huggingface_dataset_name, max_length):
        self.dataset = load_dataset(huggingface_dataset_name)
        self.tokenizer = SentenceTransformer('all-MiniLM-L6-v2', device="mps")
        self.max_length = max_length

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item["text"]
        label = item["label"]
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        return encoding["input_ids"].squeeze(), encoding["attention_mask"].squeeze(), torch.tensor(label)

    def __len__(self):
        return len(self.dataset)


tokenizing the splits (num_proc=7):   0%|          | 0/65118 [00:00<?, ? examples/s]